In [1]:
from unsloth import FastLanguageModel
import torch
import json

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
models = [
    "unsloth/Mistral-Small-24B-Base-2501",
    "unsloth/Mistral-Small-24B-Base-2501-bnb-4bit",
    "unsloth/Mistral-Small-24B-Base-2501-unsloth-bnb-4bit",
    "unsloth/Mistral-Small-24B-Instruct-2501",
    "unsloth/Mistral-Small-24B-Instruct-2501-bnb-4bit",
    "unsloth/Mistral-Small-24B-Instruct-2501-unsloth-bnb-4bit",
    "unsloth/phi-4",
    "unsloth/phi-4-bnb-4bit",
    "unsloth/phi-4-unsloth-bnb-4bit",
] 

dtype = None
load_in_4bit = False
max_seq_length = 8192

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = models[3],
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.1.8: Fast Mistral patching. Transformers: 4.48.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/29.9k [00:00<?, ?B/s]

model-00001-of-00010.safetensors:   0%|          | 0.00/4.78G [00:00<?, ?B/s]

model-00002-of-00010.safetensors:   0%|          | 0.00/4.78G [00:00<?, ?B/s]

model-00003-of-00010.safetensors:   0%|          | 0.00/4.78G [00:00<?, ?B/s]

...

model-00006-of-00010.safetensors:   0%|          | 0.00/4.78G [00:00<?, ?B/s]

model-00007-of-00010.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00008-of-00010.safetensors:   0%|          | 0.00/4.78G [00:00<?, ?B/s]

/usr/local/lib/python3.12/site-packages/unsloth/models/llama.py:1599: SyntaxWarning: invalid escape sequence '\ '
  f"   \\\   /|    GPU: {gpu_stats.name}. Max memory: {max_memory} GB. Platform: {platform_system}.\n"\
/usr/local/lib/python3.12/site-packages/unsloth/models/llama.py:1600: SyntaxWarning: invalid escape sequence '\_'
  f"O^O/ \_/ \\    Torch: {torch.__version__}. CUDA: {gpu_stats.major}.{gpu_stats.minor}. CUDA Toolkit: {torch.version.cuda}. Triton: {triton_version}\n"\
/usr/local/lib/python3.12/site-packages/unsloth/models/llama.py:1601: SyntaxWarning: invalid escape sequence '\ '
  f"\        /    Bfloat16 = {str(SUPPORTS_BFLOAT16).upper()}. FA [Xformers = {xformers_version}. FA2 = {HAS_FLASH_ATTENTION}]\n"\
/usr/local/lib/python3.12/site-packages/unsloth/models/llama.py:1752: SyntaxWarning: invalid escape sequence '\.'
  start = re.search('logger\.info\([\"\'].+?Running training', inner_training_loop).span(0)[0]
/usr/local/lib/python3.12/site-packages/unsloth/models/llam

KeyboardInterrupt: 

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.1.8 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


In [4]:
from unsloth.chat_templates import get_chat_template
"""
#Phi
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-4",
)

"""
# Mistral
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True,
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [
        tokenizer.apply_chat_template(
            convo, tokenize = False, add_generation_prompt = False
        )
        for convo in convos
    ]
    return { "text" : texts, }
pass

Unsloth: Will map <|im_end|> to EOS = </s>.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


Note that all the conversions of format are a bit pointless, I'm just doing them to be more faithful from to the original unsloth notebooks and to avoid regenerating
syntetic data. It would be possible to just have the llm output the right format in json. Or use get_chat_template on the format I have.

In [5]:
def convert_conversations_json(input_file, output_file):
    """
    Converts conversation JSON format between files
    Input format: {"conversations": [[{"conversations": [{"role": "...", "content": "..."}]}]]}
    Output format: {'conversations': [[{'from': '...', 'value': '...'}]]}
    """
    try:
        # Read input JSON
        with open(input_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            
        result = []
        
        # Process each conversation group in the conversations list
        for conv_group in data['conversations']:
            messages = []
            
            # Extract the conversations from the nested structure
            conv_data = conv_group[0]['conversations']
            
            # Process each message in the conversation
            for msg in conv_data:
                # Map 'role' to 'from' and handle 'assistant' -> 'gpt' conversion
                from_value = 'gpt' if msg['role'] == 'assistant' else msg['role']
                
                converted_msg = {
                    'from': from_value,
                    'value': msg['content']
                }
                messages.append(converted_msg)
                
            result.append(messages)
        
        # Create output JSON
        output = {'conversations': result}
        
        # Write to output file
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(output, f, indent=2, ensure_ascii=False)
            
        return f"Successfully converted {input_file} to {output_file}"
        
    except Exception as e:
        return f"Error converting JSON: {str(e)}"

In [6]:
convert_conversations_json('/notebooks/AlphaBuffet/Dataset/Processed/Ground Truth/dataset_combined_2.json',
                           '/notebooks/AlphaBuffet/Dataset/Processed/Ground Truth/dataset_clean.json')

'Successfully converted /notebooks/AlphaBuffet/Dataset/Processed/Ground Truth/dataset_combined_2.json to /notebooks/AlphaBuffet/Dataset/Processed/Ground Truth/dataset_clean.json'

In [7]:
from datasets import load_dataset
dataset = load_dataset(
    "json",
    data_files = "/notebooks/AlphaBuffet/Dataset/Processed/Ground Truth/dataset_clean.json",
    split = "train",
)

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
dataset[5]['conversations']

[{'from': 'human',
  'value': 'How did the overall performance of your equity investments compare to the Dow-Jones Industrial Average during the same period?'},
 {'from': 'gpt',
  'value': "Well, it's pretty clear we came out on top. While the Dow-Jones Industrial Average actually declined from 852 to 805, our overall unrealized and realized pre-tax gains in equities for the three-year period came to approximately $112 million. I'd say that was a marvelous period for us value-oriented equity buyers."}]

In [9]:
from unsloth.chat_templates import standardize_sharegpt

dataset = standardize_sharegpt(dataset)

Standardizing format:   0%|          | 0/3482 [00:00<?, ? examples/s]

In [10]:
dataset[5]['conversations']

[{'content': 'How did the overall performance of your equity investments compare to the Dow-Jones Industrial Average during the same period?',
  'role': 'user'},
 {'content': "Well, it's pretty clear we came out on top. While the Dow-Jones Industrial Average actually declined from 852 to 805, our overall unrealized and realized pre-tax gains in equities for the three-year period came to approximately $112 million. I'd say that was a marvelous period for us value-oriented equity buyers.",
  'role': 'assistant'}]

In [11]:
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

Map:   0%|          | 0/3482 [00:00<?, ? examples/s]

In [12]:
dataset[5]['text']

"<|im_start|>user\nHow did the overall performance of your equity investments compare to the Dow-Jones Industrial Average during the same period?<|im_end|>\n<|im_start|>assistant\nWell, it's pretty clear we came out on top. While the Dow-Jones Industrial Average actually declined from 852 to 805, our overall unrealized and realized pre-tax gains in equities for the three-year period came to approximately $112 million. I'd say that was a marvelous period for us value-oriented equity buyers.<|im_end|>\n"

In [18]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 2,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Map (num_proc=2):   0%|          | 0/3482 [00:00<?, ? examples/s]

In [19]:
from unsloth.chat_templates import train_on_responses_only
"""
#Phi

trainer = train_on_responses_only(
    trainer,
    instruction_part="<|im_start|>user<|im_sep|>",
    response_part="<|im_start|>assistant<|im_sep|>",
)

"""
# Mistral

trainer = train_on_responses_only(
    trainer,
    instruction_part="<|im_start|>user\n",
    response_part="<|im_start|>assistant\n",
)


Map:   0%|          | 0/3482 [00:00<?, ? examples/s]

In [20]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

"<|im_start|>user\nHow did the overall performance of your equity investments compare to the Dow-Jones Industrial Average during the same period?<|im_end|>\n<|im_start|>assistant\nWell, it's pretty clear we came out on top. While the Dow-Jones Industrial Average actually declined from 852 to 805, our overall unrealized and realized pre-tax gains in equities for the three-year period came to approximately $112 million. I'd say that was a marvelous period for us value-oriented equity buyers.<|im_end|>\n"

In [21]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

"                                       \nWell, it's pretty clear we came out on top. While the Dow-Jones Industrial Average actually declined from 852 to 805, our overall unrealized and realized pre-tax gains in equities for the three-year period came to approximately $112 million. I'd say that was a marvelous period for us value-oriented equity buyers.<|im_end|>\n"

In [22]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA RTX A6000. Max memory = 47.536 GB.
46.035 GB of memory reserved.


In [23]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,482 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 870
 "-____-"     Number of trainable parameters = 184,811,520


Step,Training Loss
1,1.354500
2,1.261500
3,1.082200
4,0.675600
5,0.846300
6,0.779400
7,0.684000
8,0.768700
9,0.733700
10,0.722300


In [ ]:
# save_method = "merged_4bit" or "merged_16bit"

model.push_to_hub_merged("AlphaBuffet", tokenizer, save_method = "merged_16bit", token = "***")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 23.1 out of 88.38 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 70%|███████   | 28/40 [01:43<00:45,  3.82s/it]

In [34]:
import torch
import gc

# Clear torch cuda cache
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

# Move model to CPU if it exists in memory
try:
    model = model.cpu()
except:
    pass